In [1]:
%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd


In [2]:
def CharactersTyped(ScratchPad):
    
    NotInclude = ['scratchPadText', '']
    PhraseTypedAll = [item[1] for item in scratchPad]

    IndexNotInclude = [itemInd for itemInd in range(0,len(PhraseTypedAll)) if PhraseTypedAll[itemInd] == '']

    PhrasesTypedEnd = [PhraseTypedAll[item-1] for item in IndexNotInclude if item != 1]
    
    return len(''.join(PhrasesTypedEnd))

In [22]:
def FrequencyScratchPad(UserLooksAtKey):
    
    ScratchPadLookedInd = [ind for ind in range(0,len(UserLooksAtKey)) if 'ScratchPad' in UserLooksAtKey[ind]]
    ScratchPadRow = [UserLooksAtKey[ind] for ind in range(0,len(UserLooksAtKey)) if 'ScratchPad' in UserLooksAtKey[ind]]
    ScratchPadIndDiff = np.diff(np.asarray(ScratchPadLookedInd))
    
    nScratchPad = 1
    
    for ind1 in range(0, len(ScratchPadIndDiff)):
        
        if ScratchPadIndDiff[ind1] > 1:
            #print('-1', ScratchPadLookedInd[ind1])
        # check if it is the same fixation        
            #print(ScratchPadRow[ind1][2], ScratchPadRow[ind1+1][2])
            
            if float(ScratchPadRow[ind1][2]) > float(ScratchPadRow[ind1+1][2]):
                nScratchPad = nScratchPad + 1
                
            elif float(ScratchPadRow[ind1][2]) == 0: # sometimes, after looking at the scratchpad, the last look has progress 0
                if ScratchPadIndDiff[ind1-1] == 1:
                    nScratchPad = nScratchPad + 1
                    
    # the last set of looking at scratchpad is not counted above
    if ScratchPadIndDiff[-1] < 2:
        nScratchPad = nScratchPad + 1
                
    return nScratchPad
        

In [23]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[1])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])
     

In [24]:
metricComputed = 'read_text_events_frequency_ratio'
dataFolderName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
scratchPadToTypingRatioFrequencyList = list()
Names = list()

for root, dirs, subfolder in os.walk(dataFolderName):

    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'notCompleted' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            if 'js' in root:
                if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                    try:
                        fScratchPad = open(root + '\\' + file)
                        readerScratchPad = csv.reader(fScratchPad)
                        scratchPad = list(readerScratchPad)
                        scratchPad.remove(scratchPad[0])
                        #print(scratchPad)
                    except:
                        if fScratchPad is not None:
                            fScratchPad.close()
                        else:
                            print('error in opening the scratchpad log file')
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
        if scratchPad is None or userKeys is None:
            continue
        else:
            
            print(root)

            # Number of times scratchpad was looked at
            nScratchPad = FrequencyScratchPad(userKeys)
                
            # Number of characters typed
            nTypedChar = CharactersTyped(scratchPad)
                
            # save only the name of the subject and the date part of the file
            a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                
            Names.append(a.findall(root)[0])
            scratchPadToTypingRatioFrequencyList.append(float(nScratchPad/nTypedChar))
                
            #print(nScratchPad, nTypedChar)
                
            nCount = nCount + 1
           
                
if scratchPadToTypingRatioFrequencyList:
    AddDataToFile(metricComputed, Names, scratchPadToTypingRatioFrequencyList)
                        

C:\DTU\Data\201805_HealthnRehab\TypingData\May15\akt_MS\OptiKeyLogs\2018-5-15-14-3-18
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\be_DT\OptiKeyLogs\2018-5-15-14-51-13-notInclude
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\jl_DT\OptiKeyLogs\2018-5-15-10-21-20
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\KEA_MS\OptiKeyLogs\2018-5-15-12-31-49
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\lone_DT\OptiKeyLogs\2018-5-15-11-4-33
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\mcc_MS\OptiKeyLogs\2018-5-15-12-7-17
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\MK_DT\OptiKeyLogs\2018-5-15-14-32-48
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\MT_MS\OptiKeyLogs\2018-5-15-10-52-23-notInclude
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\ok_MS\OptiKeyLogs\2018-5-15-12-23-16
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\pt_DT\OptiKeyLogs\2018-5-15-12-43-13
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\sc_MS\OptiKeyLogs\2018-5-15-14-25-39
C:\DTU\Data\201805_HealthnReha